In [6]:
!python -m spacy download fr_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 40.1 MB/s eta 0:00:00:00:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
pip install PyPDF2 PyMuPDF pdfplumber spacy

In [ ]:
import spacy
import pdfplumber

nlp = spacy.load("fr_core_news_md")

financial_terms = {
    "dette nette": "NetDebt",
    "Résultat opérationnel": "OperatingProfit",
    "actifs financiers": "FinancialAssets",
    "passifs courants": "CurrentLiabilities"
}

def preprocess_bio_tagging(text):
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents]
    tagged_sentences = []
    
    for sentence in sentences:
        tokens = sentence.split()
        tags = ["O"] * len(tokens)
        
        for term, tag in financial_terms.items():
            term_tokens = term.split()
            if ' '.join(tokens).find(term) != -1:
                start_idx = tokens.index(term_tokens[0])
                tags[start_idx] = f"B-{tag}"
                for i in range(1, len(term_tokens)):
                    tags[start_idx + i] = f"I-{tag}"
        
        tagged_sentence = list(zip(tokens, tags))
        tagged_sentences.append(tagged_sentence)

    return tagged_sentences

pdf_path = "/kaggle/input/fleury-michon/fleury_michon_-_rapport_sur_les_comptes_semestriels_2023.pdf"

with pdfplumber.open(pdf_path) as pdf:
    pdf_text = ""
    for page in pdf.pages:
        pdf_text += page.extract_text()

tagged_data = preprocess_bio_tagging(pdf_text)

for sentence in tagged_data:
    print(sentence)


In [10]:
import fitz  # PyMuPDF

def annotate_pdf(input_pdf_path, output_pdf_path, tagged_data):
    pdf_document = fitz.open(input_pdf_path)

    page_num = 0
    for page in pdf_document:
        text = page.get_text("text")
        tagged_sentence = tagged_data[page_num]  # Match tagged sentences with corresponding PDF page
        
        for word, tag in tagged_sentence:
            if tag != "O":
                text_instances = page.search_for(word)  # Locate the word in the text
                if text_instances:
                    for inst in text_instances:
                        # Highlight or underline the word
                        page.add_underline_annot(inst)

        page_num += 1

    pdf_document.save(output_pdf_path)

# Extract text and process for tagging
pdf_path = "/kaggle/input/fleury-michon/fleury_michon_-_rapport_sur_les_comptes_semestriels_2023.pdf"
output_path = "annotated_output.pdf"

with pdfplumber.open(pdf_path) as pdf:
    pdf_text = ""
    for page in pdf.pages:
        pdf_text += page.extract_text()

tagged_data = preprocess_bio_tagging(pdf_text)

# Annotate the PDF
annotate_pdf(pdf_path, output_path, tagged_data)


In [10]:
import fitz  # PyMuPDF
import spacy
import pdfplumber

# Load SpaCy model
nlp = spacy.load("fr_core_news_md")

# Define the financial terms for tagging
financial_terms = {
    "dette nette": "NetDebt",
    "Résultat opérationnel": "OperatingProfit",
    "actifs financiers": "FinancialAssets",
    "passifs courants": "CurrentLiabilities"
}

# Preprocess function to apply BIO tagging
def preprocess_bio_tagging(text):
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents]
    tagged_sentences = []
    
    for sentence in sentences:
        tokens = sentence.split()
        tags = ["O"] * len(tokens)
        
        for term, tag in financial_terms.items():
            term_tokens = term.split()
            if ' '.join(tokens).find(term) != -1:
                start_idx = tokens.index(term_tokens[0])
                tags[start_idx] = f"B-{tag}"
                for i in range(1, len(term_tokens)):
                    tags[start_idx + i] = f"I-{tag}"
        
        tagged_sentence = list(zip(tokens, tags))
        tagged_sentences.append(tagged_sentence)

    return tagged_sentences

# Annotate the PDF based on BIO tagging
def annotate_pdf(input_pdf_path, output_pdf_path, tagged_data):
    pdf_document = fitz.open(input_pdf_path)
    
    page_num = 0
    for page in pdf_document:
        text = page.get_text("text")
        tagged_sentence = tagged_data[page_num]  # Get the tagged data for the current page
        
        # Tokenize PDF text to align with tagged data
        page_tokens = text.split()
        
        for i, (word, tag) in enumerate(tagged_sentence):
            if tag != "O":
                # We need to account for any variations in tokenization here
                word_instance = page_tokens[i]
                text_instances = page.search_for(word_instance)  # Search for the word in the PDF
                
                if text_instances:
                    for inst in text_instances:
                        # Add highlight annotation
                        page.add_highlight_annot(inst)

        page_num += 1

    # Save the annotated PDF with highlighted text
    pdf_document.save(output_pdf_path)

# Extract and process PDF text
pdf_path = "/kaggle/input/fleury-michon/fleury_michon_-_rapport_sur_les_comptes_semestriels_2023.pdf"
output_path = "annotated_out.pdf"

with pdfplumber.open(pdf_path) as pdf:
    pdf_text = ""
    for page in pdf.pages:
        pdf_text += page.extract_text()

# Preprocess the extracted text and apply BIO tagging
tagged_data = preprocess_bio_tagging(pdf_text)

# Annotate the PDF with tagged data
annotate_pdf(pdf_path, output_path, tagged_data)
